In [1]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 122.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.6 MB/s eta 0:00:00


In [2]:
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

# Download Pre-trained Model from Huggingface
bert_model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

# Tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
question = '''what is a brain tumor'''

paragraph = '''

A brain tumor is an abnormal growth or mass of cells that can develop within the brain. These tumors have diverse origins, potentially arising from various types of brain cells, including glial cells that provide support and insulation for nerve cells, or even from the neurons themselves. Brain tumors are broadly categorized as either benign (non-cancerous) or malignant (cancerous). They can manifest as primary tumors, originating within the brain, or secondary tumors, which spread to the brain from other parts of the body.

The process of diagnosing and comprehending brain tumors often involves the utilization of advanced imaging techniques, with brain scans serving as indispensable tools. Among the primary imaging methods, Magnetic Resonance Imaging (MRI) and Computed Tomography (CT) scans stand out for their ability to visualize brain structures and identify abnormalities, including tumors.

MRI scans employ powerful magnetic fields and radio waves to generate intricate and detailed images of the brain's soft tissues. They offer exceptional contrast, making it possible to differentiate between healthy brain tissue and tumor masses. In contrast, CT scans rely on X-rays to produce cross-sectional images of the brain. Although CT scans are faster and more readily available, they may not provide the same level of detail as MRI scans.

The interpretation of brain scans necessitates expertise, often from specialists such as neuroradiologists or neurologists. When scrutinizing brain scans, it's vital to pay close attention to the precise location, size, shape, and specific characteristics of any identified abnormality. Tumors frequently present themselves as masses with irregular shapes and may exhibit varying degrees of contrast enhancement. Factors like the involvement of surrounding brain tissue and indications of edema (swelling) also play pivotal roles in accurate diagnosis and assessment.

Moreover, understanding the precise type of brain tumor is paramount for devising effective treatment strategies. Brain tumors can encompass a spectrum of classifications, including gliomas, meningiomas, metastatic tumors, or other distinct types, each characterized by its unique attributes and prognostic considerations. Achieving this level of specificity may necessitate specialized imaging techniques and, in some cases, a biopsy to ascertain an accurate classification.




'''


# Encode the question and paragraph using BERT tokenizer
encoding = bert_tokenizer.encode_plus(text=question, text_pair=paragraph)

# Token embeddings
token_ids = encoding['input_ids']
# input tokens
tokens = bert_tokenizer.convert_ids_to_tokens(token_ids)

# Segment embeddings
sentence_embedding = encoding['token_type_ids']

# Pass the encoded input to the BERT model
bert_out = bert_model(torch.tensor([token_ids]), token_type_ids=torch.tensor([sentence_embedding]))

# Retrieve the start and end indices of the answer
start_index = torch.argmax(bert_out['start_logits'])

end_index = torch.argmax(bert_out['end_logits'])

print('start_index: ', start_index, 'end_index: ', end_index)

# Extract the predicted answer
answer = ' '.join(tokens[start_index:end_index + 1])

# Clean the answer
final_answer = ''

for word in answer.split():

    # If it's a subword token
    if word[0:2] == '##':
        final_answer += word[2:]

    elif answer.startswith("[CLS]") or answer.startswith("[SEP]"):
        final_answer = "Unable to find the answer to your question."

    else:
        final_answer += ' ' + word

print(final_answer)

start_index:  tensor(11) end_index:  tensor(23)
 an abnormal growth or mass of cells that can develop within the brain
